In [1]:
"""add hidden layer"""
import numpy as np
from tensorflow_core.python.keras.api._v2.keras.datasets import mnist
from tensorflow_core.python.keras.api._v2.keras.models import Sequential
from tensorflow_core.python.keras.api._v2.keras.layers import Dense, Activation
from tensorflow_core.python.keras.api._v2.keras.optimizers import SGD
from tensorflow_core.python.keras.api._v2.keras.utils import to_categorical
from make_tensorboard import make_tensorboard

In [2]:
# 乱数の固定
np.random.seed(1671)

In [3]:
# パラメータ設定
NB_EPOCH = 20           # エポック数
BATCH_SIZE = 128        # バッチサイズ
VERBOSE = 1             #
NB_CLASSES = 10         # 出力数
OPTIMIZER = SGD()       # 最適化関数
N_HIDDEN = 128          # 隠れ層数
VALIDATION_SPLIT = 0.2  # 検証データの割合

In [4]:
# データの読み込み
(X_train, y_train), (X_test, y_test) = mnist.load_data()
RESHAPED = 784  # 28×28 = 784

In [5]:
# データの整形
TRAIN_DATA_SIZE = 60000
TEST_DATA_SIZE = 10000
X_train = X_train.reshape(TRAIN_DATA_SIZE, RESHAPED)
X_test = X_test.reshape(TEST_DATA_SIZE, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [6]:
# 正規化
X_train /= 255
X_test /= 255

print(X_train.shape[0], 'train sample')
print(X_test.shape[0], 'test sample')

60000 train sample
10000 test sample


In [7]:
# ラベルをOne-hotエンコーディング
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)

In [8]:
# Sequentialモデルを定義
model = Sequential()
# 層を定義
model.add(Dense(N_HIDDEN, input_shape=(RESHAPED,)))  # ← 784 × 128
model.add(Activation('relu'))
model.add(Dense(N_HIDDEN))                           # ← 128 × 128
model.add(Activation('relu'))
model.add(Dense(NB_CLASSES))                         # ← 128 × 10
model.add(Activation('softmax'))
# モデルを評価しTensorBoard互換のログに書き込む
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
__________________________________________________

In [9]:
# モデルをコンパイル
# -損失関数　：カテゴリカルクロスエントロピー
# -最適化関数：確率的勾配降下法
# -評価関数　：Accuracy(正解率)
model.compile(loss='categorical_crossentropy',
              optimizer=OPTIMIZER,
              metrics=['accuracy'])

In [10]:
# tensorboard用
callbacks = [make_tensorboard(set_dir_name='keras_MINST_V2')]

In [11]:
# 学習
# -バッチサイズ　　：128
# -エポック数　　　：200
# -コールバック関数：make_tensorboard(訓練の各段階で呼び出される関数)
# -ＶＥＲＢＯＳＥ　：詳細表示モードON
# -検証データの割合：20%
model.fit(X_train, Y_train,
          batch_size=BATCH_SIZE,
          epochs=NB_EPOCH,
          callbacks=callbacks,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 3s 56us/sample - loss: 1.4223 - accuracy: 0.6689 - val_loss: 0.6984 - val_accuracy: 0.8485
Epoch 2/20
48000/48000 [==============================] - 1s 27us/sample - loss: 0.5681 - accuracy: 0.8575 - val_loss: 0.4375 - val_accuracy: 0.8842
Epoch 3/20
48000/48000 [==============================] - 1s 27us/sample - loss: 0.4262 - accuracy: 0.8823 - val_loss: 0.3647 - val_accuracy: 0.8999
Epoch 4/20
48000/48000 [==============================] - 1s 27us/sample - loss: 0.3726 - accuracy: 0.8942 - val_loss: 0.3308 - val_accuracy: 0.9057
Epoch 5/20
48000/48000 [==============================] - 1s 27us/sample - loss: 0.3412 - accuracy: 0.9030 - val_loss: 0.3077 - val_accuracy: 0.9123
Epoch 6/20
48000/48000 [==============================] - 1s 26us/sample - loss: 0.3194 - accuracy: 0.9084 - val_loss: 0.2913 - val_accuracy: 0.9173
Epoch 7/20
48000/48000 [==============================] 

In [12]:
# スコアを表示
score = model.evaluate(X_test, Y_test, verbose=0)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])


Test score: 0.18869757368266582
Test accuracy: 0.9465
